In [1]:
import os
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import pypose as pp
import numpy as np
from utile import parse_param
from nn_utile import AUVTraj, AUVStep, AUVRNNDeltaV

import warnings

In [34]:
import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
opt_mod = torch.compile(mod)
print(opt_mod(torch.randn(10, 100)))

/tmp/ipykernel_18212/1297660278.py:11: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(
[2023-05-03 17:25:41,933] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward
[2023-05-03 17:25:41,934] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_18212/1297660278.py:22
[2023-05-03 17:25:41,934] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_GLOBAL torch []
[2023-05-03 17:25:41,935] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR nn [TorchVariable(<module 'torch' from '/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/__init__.py'>)]
[2023-05-03 17:25:41,935] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR functional [TorchVariable(<module 'torch.nn' from '/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/nn/__init__.py'>)]
[2023-05-03 17:25:41,936] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR relu [TorchV

BackendCompilerFailed: debug_wrapper raised DataDependentOutputException: aten._local_scalar_dense.default

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [30]:
import torch._dynamo
import logging
torch._dynamo.logging.set_loggers_level(logging.DEBUG)


class MyModule(torch.nn.Module):
    def __init__(self, in_size=10, out_size=10):
        super(MyModule, self).__init__()
        self.fc0 = torch.nn.Linear(in_size, 100)
        self.fc1 = torch.nn.Linear(100, out_size)

    def forward(self, x):
        fc0 = self.fc0(x)
        return fc0

size_in, size_out=100, 100
m = MyModule(size_in, size_out)

compiled_m = torch.compile(m)


a = torch.rand(100, size_in).to(device="cuda:0")

In [31]:
for i in range(100):
    compiled_m(a)

[2023-05-03 17:21:17,798] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward
[2023-05-03 17:21:17,799] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /tmp/ipykernel_18212/4199708273.py:19
[2023-05-03 17:21:17,799] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST self []
[2023-05-03 17:21:17,800] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR fc0 [NNModuleVariable()]
[2023-05-03 17:21:17,802] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [NNModuleVariable()]
[2023-05-03 17:21:17,802] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [NNModuleVariable(), TensorVariable()]


TorchRuntimeError: 

from user code:
   File "/tmp/ipykernel_18212/4199708273.py", line 19, in forward
    fc0 = self.fc0(x)

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [12]:
def get_device(gpu=False, unit=0):
    use_cuda = False
    if gpu:
        use_cuda = torch.cuda.is_available()
        if not use_cuda:
            warnings.warn("Asked for GPU but torch couldn't find a Cuda capable device")
    return torch.device(f"cuda:{unit}" if use_cuda else "cpu")

'''
Runs a given RNN Model with a given input state and action sequence.
'''
def run(model, state, X):
    # TODO: disable all log, just keep the trajectory.
    traj = model(state, X)
    return traj

'''
Load a RNN Model given a checkpoint file.
'''
def load_model(model, ckpt_path):
    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt)
    return model


def create_model(params, device):
    model = AUVTraj(params).to(device)
    return model

In [13]:
path = "../train_log/2023.04.24-11:25:38/"
param_file = os.path.join(path, "parameters.yaml")
param = parse_param(param_file)
ckpt_path = os.path.join(path, "ckpt.pth")

In [14]:
device = get_device(True)

state = torch.zeros(size=(20, 1, 13)).to(device)
state[..., 6] = 1.
seq = torch.zeros(size=(20, 10, 6)).to(device)

In [15]:

dv = AUVRNNDeltaV().to(device)
h = None
p = state[..., :7]
v = state[..., 7:]

x = pp.SE3(p).to(device)
# Warm-up
print("Compiling...")
compiled_dv = torch.compile(dv)
torch.no_grad()
print("Done")


print("Warm-start")
res, h = compiled_dv(x, v, seq[:, 0:1], h)

with profile(with_stack=True, profile_memory=True, use_cuda=True) as prof:
    with record_function("model_inference"):
        res, h = compiled_dv(x, v, seq[:, 0:1], h)

print(prof.key_averages(group_by_stack_n=10).table(sort_by="self_cpu_time_total", row_limit=5))

[2023-05-03 15:53:07,255] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing <graph break in forward>
[2023-05-03 15:53:07,255] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST ___stack0 []
[2023-05-03 15:53:07,256] torch._dynamo.symbolic_convert: [DEBUG] TRACE JUMP_ABSOLUTE 24 [TensorVariable()]
[2023-05-03 15:53:07,256] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR flatten [TensorVariable()]
[2023-05-03 15:53:07,257] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CONST -2 [GetAttrVariable(TensorVariable(), flatten)]
[2023-05-03 15:53:07,258] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_CONST ('start_dim',) [GetAttrVariable(TensorVariable(), flatten), ConstantVariable(int)]
[2023-05-03 15:53:07,259] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION_KW 1 [GetAttrVariable(TensorVariable(), flatten), ConstantVariable(int), ConstantVariable(tuple)]
[2023-05-03 15:53:07,262] torch._dynamo.symbolic_convert: [DEBUG] TRACE STORE_FAST 

Compiling...
Done
Warm-start


[2023-05-03 15:53:07,468] torch._dynamo.symbolic_convert: [DEBUG] TRACE STORE_FAST input [ListIteratorVariable(), TensorVariable()]
[2023-05-03 15:53:07,469] torch._dynamo.symbolic_convert: [DEBUG] TRACE JUMP_ABSOLUTE 4 [ListIteratorVariable()]
[2023-05-03 15:53:07,469] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/nn/modules/container.py:216
[2023-05-03 15:53:07,470] torch._dynamo.symbolic_convert: [DEBUG] TRACE FOR_ITER 18 [ListIteratorVariable()]
[2023-05-03 15:53:07,470] torch._dynamo.symbolic_convert: [DEBUG] TRACE STORE_FAST module [ListIteratorVariable(), NNModuleVariable()]
[2023-05-03 15:53:07,471] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/nn/modules/container.py:217
[2023-05-03 15:53:07,471] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST module [ListIteratorVariable()]
[2023-05-03 15:53:07,472] torch

BackendCompilerFailed: debug_wrapper raised DataDependentOutputException: aten._local_scalar_dense.default

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [6]:
step = AUVStep().to(device)
h = None
p = state[..., :7]
v = state[..., 7:]

x = pp.SE3(p).to(device)

# Warm-up
print("Compiling...")
compiled_step = torch.compile(step)
print("Done")
x, v, dv, h = compiled_step(x, v, seq[:, 0:1], h)

with profile(with_stack=True, profile_memory=True, use_cuda=True) as prof:
    with record_function("model_inference"):
        compiled_step(x, v, seq[:, 0:1], h)

print(prof.key_averages(group_by_stack_n=10).table(sort_by="self_cpu_time_total", row_limit=5))

/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/autograd/__init__.py:303: UserWarning: Error detected in MmBackward0. Traceback of forward call that caused the error:
  File "/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/nn/modules/container.py", line 217, in forward
    input = module(input)
 (Triggered internally at ../torch/csrc/autograd/python_anomaly_mode.cpp:114.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


BackendCompilerFailed: debug_wrapper raised DataDependentOutputException: aten._local_scalar_dense.default

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [5]:
model = create_model(param, device)
model = load_model(model, ckpt_path)
# Warm-up
compiled_model = torch.compile(model)
traj = compiled_model(state, seq)

with profile(with_stack=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        compiled_model(state, seq)

/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/autograd/__init__.py:303: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "/home/pierre/anaconda3/envs/pp_local/lib/python3.9/site-packages/torch/nn/modules/container.py", line 217, in forward
    input = module(input)
 (Triggered internally at ../torch/csrc/autograd/python_anomaly_mode.cpp:114.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


BackendCompilerFailed: debug_wrapper raised DataDependentOutputException: aten._local_scalar_dense.default

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True
